### Configuration et préparation des données

In [ ]:
import dspy
import warnings
warnings.filterwarnings('ignore')

# Configurer le modèle de langage
lm = dspy.LM(
    model='ollama_chat/llama3.1:8b',
    api_base='http://localhost:11434',
    temperature=0.3
)

# Configurer DSPy globalement
dspy.configure(lm=lm)

In [ ]:
# Données d'entraînement
trainset = [
    {"ticket": "Mon ordinateur ne démarre plus depuis ce matin. J'ai une présentation importante dans 2 heures.", "category": "Hardware", "priority": "Urgent"},
    {"ticket": "Je n'arrive pas à me connecter à l'imprimante du 3e étage. Ça peut attendre.", "category": "Peripherals", "priority": "Low"},
    {"ticket": "Le VPN ne fonctionne plus. Impossible d'accéder aux fichiers du serveur.", "category": "Network", "priority": "High"},
    {"ticket": "J'ai oublié mon mot de passe Outlook. Je peux utiliser le webmail.", "category": "Account", "priority": "Medium"},
    {"ticket": "Le site web affiche une erreur 500. Les clients ne peuvent plus commander!", "category": "Application", "priority": "Critical"},
    {"ticket": "Ma souris sans fil ne répond plus bien. Les piles sont faibles.", "category": "Peripherals", "priority": "Low"},
    {"ticket": "Le système de paie ne calcule pas les heures supplémentaires. C'est la fin du mois.", "category": "Application", "priority": "Urgent"},
    {"ticket": "J'aimerais une mise à jour de mon logiciel Adobe quand vous aurez le temps.", "category": "Software", "priority": "Low"},
    {"ticket": "Le serveur de base de données est très lent. Toute la production est impactée.", "category": "Infrastructure", "priority": "Critical"},
    {"ticket": "Je ne reçois plus les emails. J'attends des réponses de fournisseurs.", "category": "Email", "priority": "High"},
    {"ticket": "Mon écran externe ne s'affiche plus. Je peux travailler sur le laptop.", "category": "Hardware", "priority": "Medium"},
    {"ticket": "Le wifi de la salle A ne fonctionne pas. Réunion avec des externes dans 30 min.", "category": "Network", "priority": "Urgent"},
    {"ticket": "Je voudrais installer Slack pour mieux collaborer avec l'équipe.", "category": "Software", "priority": "Medium"},
    {"ticket": "Le système de sauvegarde a échoué cette nuit selon le rapport.", "category": "Infrastructure", "priority": "High"},
    {"ticket": "Mon clavier a une touche qui colle. C'est gérable mais ennuyeux.", "category": "Peripherals", "priority": "Low"}
]

# Données de validation
valset = [
    {"ticket": "Le serveur de fichiers est inaccessible. Personne ne peut travailler.", "category": "Infrastructure", "priority": "Critical"},
    {"ticket": "J'ai besoin d'accès au dossier comptabilité pour l'audit. C'est urgent.", "category": "Account", "priority": "Urgent"},
    {"ticket": "L'écran de mon collègue en vacances clignote. On peut attendre.", "category": "Hardware", "priority": "Low"},
    {"ticket": "Le CRM plante quand j'essaie d'exporter les contacts.", "category": "Application", "priority": "High"},
    {"ticket": "Je voudrais changer ma photo de profil quand vous aurez un moment.", "category": "Account", "priority": "Low"},
    {"ticket": "La vidéoconférence ne fonctionne pas. Réunion avec New York dans 10 minutes!", "category": "Application", "priority": "Critical"},
    {"ticket": "Mon antivirus affiche un message d'expiration mais tout fonctionne.", "category": "Software", "priority": "Medium"}
]

# Catégories et priorités possibles
CATEGORIES = ["Hardware", "Software", "Network", "Application", "Infrastructure", "Account", "Email", "Peripherals"]
PRIORITIES = ["Low", "Medium", "High", "Urgent", "Critical"]

print(f"📊 Données chargées : {len(trainset)} entraînement, {len(valset)} validation")
print(f"📦 {len(CATEGORIES)} catégories, {len(PRIORITIES)} priorités")

# Partie 4 : L'évaluation

## Pourquoi évaluer ?

Jusqu'à présent, nous avons créé des modules et observé leurs sorties qualitativement. Mais pour :
- **Comparer** différents modules
- **Mesurer** les améliorations
- **Optimiser** automatiquement (avec GEPA)

...nous avons besoin de **mesures quantitatives** : les **métriques**.

## Qu'est-ce qu'une métrique ?

Une **métrique** est une fonction qui prend :
- Un **exemple** avec la vraie réponse (ground truth)
- Une **prédiction** du modèle
- Et retourne un **score** (généralement entre 0 et 1)

### Format d'une métrique

```python
def ma_metrique(example, prediction, trace=None, pred_name=None, pred_trace=None):
    # Comparer example et prediction
    # Retourner un score entre 0 et 1
    return score
```

**Note** : Les paramètres `trace`, `pred_name` et `pred_trace` sont optionnels et utilisés par certains optimiseurs.

## Métrique 1 : exact match (correspondance exacte)

La métrique la plus stricte : tout doit être parfait.

In [55]:
def exact_match_metric(example, prediction, trace=None, pred_name=None, pred_trace=None):
    """
    Métrique stricte : 1 si catégorie ET priorité correctes, 0 sinon
    """
    # Normaliser les chaînes (minuscules, sans espaces)
    pred_category = prediction.category.strip().lower()
    true_category = example['category'].strip().lower()
    
    pred_priority = prediction.priority.strip().lower()
    true_priority = example['priority'].strip().lower()
    
    # Les deux doivent être corrects
    if pred_category == true_category and pred_priority == true_priority:
        return 1.0
    else:
        return 0.0

# Test de la métrique
print("🎯 Métrique : exact match")
print("\nTest 1 : Prédiction correcte")
example1 = {"ticket": "Test", "category": "Hardware", "priority": "High"}
pred1 = dspy.Prediction(category="Hardware", priority="High")
score1 = exact_match_metric(example1, pred1)
print(f"  Attendu: Hardware | High")
print(f"  Prédit:  Hardware | High")
print(f"  Score: {score1}")

print("\nTest 2 : Catégorie correcte, priorité incorrecte")
pred2 = dspy.Prediction(category="Hardware", priority="Low")
score2 = exact_match_metric(example1, pred2)
print(f"  Attendu: Hardware | High")
print(f"  Prédit:  Hardware | Low")
print(f"  Score: {score2}")

🎯 Métrique : exact match

Test 1 : Prédiction correcte
  Attendu: Hardware | High
  Prédit:  Hardware | High
  Score: 1.0

Test 2 : Catégorie correcte, priorité incorrecte
  Attendu: Hardware | High
  Prédit:  Hardware | Low
  Score: 0.0


## Métrique 2 : partial match (correspondance partielle)

Plus nuancée : donne des points partiels si au moins un champ est correct.

In [56]:
def partial_match_metric(example, prediction, trace=None, pred_name=None, pred_trace=None):
    """
    Métrique nuancée avec points partiels :
    - 1.0 : Les deux corrects
    - 0.7 : Catégorie correcte uniquement
    - 0.5 : Priorité correcte uniquement
    - 0.0 : Aucun correct
    """
    pred_category = prediction.category.strip().lower()
    true_category = example['category'].strip().lower()
    
    pred_priority = prediction.priority.strip().lower()
    true_priority = example['priority'].strip().lower()
    
    category_match = (pred_category == true_category)
    priority_match = (pred_priority == true_priority)
    
    if category_match and priority_match:
        return 1.0
    elif category_match:
        return 0.7  # La catégorie est plus importante
    elif priority_match:
        return 0.5
    else:
        return 0.0

# Test de la métrique
print("🎯 Métrique : partial match")
print("\nTest 1 : Les deux corrects")
score1 = partial_match_metric(example1, pred1)
print(f"  Score: {score1}")

print("\nTest 2 : Catégorie correcte uniquement")
score2 = partial_match_metric(example1, pred2)
print(f"  Score: {score2}")

print("\nTest 3 : Priorité correcte uniquement")
pred3 = dspy.Prediction(category="Software", priority="High")
score3 = partial_match_metric(example1, pred3)
print(f"  Score: {score3}")

print("\nTest 4 : Aucun correct")
pred4 = dspy.Prediction(category="Software", priority="Low")
score4 = partial_match_metric(example1, pred4)
print(f"  Score: {score4}")

🎯 Métrique : partial match

Test 1 : Les deux corrects
  Score: 1.0

Test 2 : Catégorie correcte uniquement
  Score: 0.7

Test 3 : Priorité correcte uniquement
  Score: 0.5

Test 4 : Aucun correct
  Score: 0.0


## Fonction d'évaluation réutilisable

Créons une fonction pour évaluer n'importe quel module sur un dataset complet.

In [57]:
def evaluate_module(module, dataset, metric, verbose=False):
    """
    Évalue un module sur un dataset complet
    
    Args:
        module: Le module DSPy à évaluer
        dataset: Liste de dictionnaires avec 'ticket', 'category', 'priority'
        metric: Fonction de métrique
        verbose: Si True, affiche les détails
    
    Returns:
        float: Score moyen (entre 0 et 1)
    """
    total_score = 0
    n_examples = len(dataset)
    
    for i, example in enumerate(dataset):
        # Prédiction
        prediction = module(ticket=example['ticket'])
        
        # Calcul du score
        score = metric(example, prediction)
        total_score += score
        
        # Affichage optionnel
        if verbose:
            print(f"Exemple {i+1}/{n_examples}")
            print(f"  Ticket: {example['ticket'][:50]}...")
            print(f"  Attendu: {example['category']} | {example['priority']}")
            print(f"  Prédit:  {prediction.category} | {prediction.priority}")
            print(f"  Score: {score}\\n")
    
    # Score moyen
    avg_score = total_score / n_examples
    return avg_score

# Test de la fonction d'évaluation
print("📊 Évaluation de ChainOfThought sur le dataset de validation")
print("="*70 + "\\n")

score = evaluate_module(cot_classifier, valset, exact_match_metric, verbose=False)
print(f"Score moyen (exact match): {score:.2%}")

score_partial = evaluate_module(cot_classifier, valset, partial_match_metric, verbose=False)
print(f"Score moyen (partial match): {score_partial:.2%}")

📊 Évaluation de ChainOfThought sur le dataset de validation
======================================================================\n
Score moyen (exact match): 57.14%
Score moyen (partial match): 67.14%


## Comparaison de modules

Maintenant que nous avons des métriques, comparons nos différents modules !

In [58]:
# Comparer tous nos modules
modules_to_compare = [
    ("Predict", predict_classifier),
    ("ChainOfThought", cot_classifier),
    ("Sequential", sequential),
    ("Validated", validated),
    ("Ensemble", ensemble)
]

print("="*70)
print("Comparaison des modules")
print("="*70 + "\\n")

results = []

for name, module in modules_to_compare:
    print(f"Évaluation de {name}...")
    score_exact = evaluate_module(module, valset, exact_match_metric)
    score_partial = evaluate_module(module, valset, partial_match_metric)
    
    results.append({
        'module': name,
        'exact': score_exact,
        'partial': score_partial
    })

print("\\n" + "="*70)
print("Résultats")
print("="*70 + "\\n")

print(f"{'Module':<20} {'Exact Match':<15} {'Partial Match':<15}")
print("-" * 50)
for r in results:
    print(f"{r['module']:<20} {r['exact']:<14.1%} {r['partial']:<14.1%}")

# Trouver le meilleur
best = max(results, key=lambda x: x['exact'])
print(f"\n🏆 Meilleur module (exact match): {best['module']} avec {best['exact']:.1%}")

Comparaison des modules
======================================================================\n
Évaluation de Predict...
Évaluation de ChainOfThought...
Évaluation de Sequential...
Évaluation de Validated...
Évaluation de Ensemble...
\n======================================================================
Résultats
======================================================================\n
Module               Exact Match     Partial Match  
--------------------------------------------------
Predict              57.1%          87.1%         
ChainOfThought       57.1%          67.1%         
Sequential           42.9%          80.0%         
Validated            57.1%          67.1%         
Ensemble             57.1%          67.1%         

🏆 Meilleur module (exact match): Predict avec 57.1%


## 💡 Bonnes pratiques pour l'évaluation

### ✅ À faire

1. **Toujours avoir un dataset de validation séparé** : Ne jamais évaluer sur les données d'entraînement
2. **Utiliser plusieurs métriques** : Exact match + partial match donnent une vue complète
3. **Tester sur des cas limites** : Tickets ambigus, très courts, très longs
4. **Documenter vos métriques** : Expliquez ce que signifie chaque score
5. **Comparer de manière équitable** : Même dataset, même métrique

### ❌ À éviter

1. **Une seule métrique** : Peut cacher des problèmes
2. **Dataset trop petit** : Minimum 20-30 exemples pour validation
3. **Ignorer les erreurs** : Analyser les échecs est crucial
4. **Sur-optimiser** : Attention au surapprentissage sur le dataset de validation

### 📊 Métriques avancées (optionnel)

Pour aller plus loin, vous pouvez calculer :
- **Précision par catégorie** : Performance sur chaque catégorie séparément
- **Matrice de confusion** : Quelles catégories sont confondues
- **Temps d'exécution** : Trade-off précision/vitesse
- **Coût** : Nombre de tokens utilisés